In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# from google.colab import files
# uploaded = files.upload()

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
class suicides(object):
    def __init__(self):
        self.df = suicides.df
        self.df_test = suicides.df_test
        self.df_all_feature_var_names = []
        self.df_test_all_feature_var_names = []
        
    _non_numerical_feature_names = []
    _numerical_feature_names = []
    _is_one_hot_encoder = 0
    _feature_names_num = []
    _is_not_import_data = 0
    
    
    df = pd.read_csv('./train1.csv')
    df_test = pd.read_csv('./test_mini1.csv')
    
    
    # 수치형 열을 추출
    def extract_numerical_features(df):
        df = df.copy()
        numerical_features = pd.Series(data=False, index=df.columns, dtype=bool)
        
        for feature in df.columns:
            if any(tuple(df[feature].map(lambda x: type(x)) == int)) or \
                any(tuple(df[feature].map(lambda x: type(x)) == float)) & \
                    (not any(tuple(df[feature].map(lambda x: type(x)) == str))):
                numerical_features[feature] = 1
    
        return numerical_features[numerical_features == 1].index
    
    # 수치형을 제외한 열을 추출
    def extract_non_numerical_features(df):
        df = df.copy()
    
        return df.select_dtypes(exclude=[np.number])
    
    # 범주형을 수치형으로 바꿔서 새로운 열 생성
    def encode_labels_in_numeric_format(df, estimated_var):
        feature_names_num = ''.join([estimated_var, '_count'])
        mask = ~df[estimated_var].isnull()
        df[feature_names_num] = df[estimated_var]
        df.loc[mask, tuple([feature_names_num])] = df[estimated_var].factorize()[0][mask[mask==1].index]
    
    # 범주형 자료 내에 있는 범주의 종류를 반환
    def label_classes(df, estimated_var):
        le = LabelEncoder()
        le.fit(df[estimated_var].values)
        
        return le.classes_
    
    # 범주형 자료 one hot encoding
    def one_hot_encoder(df, estimated_var):
        
        df_class = df.copy()
        ohe = OneHotEncoder()
        label_classes = df_class[estimated_var].factorize()[1]
        new_one_hot_encoded_features = [''.join([estimated_var, '_', x]) for x in label_classes]
        mask = ~df[estimated_var].isnull()
        feature_var_values = ohe.fit_transform(np.reshape(np.array(df[''.join([estimated_var, '_count'])][mask].values), 
                                                      (df[mask].shape[0], 1))).toarray().astype(int)

        for ite in new_one_hot_encoded_features:
            df[ite] = df[estimated_var]
        df.loc[mask, tuple(new_one_hot_encoded_features)] = feature_var_values
    
    def features_with_null_logical(df, axis=1):
        
        row_length = len(df._get_axis(0))
        aggregate_axis = 1 - axis
        features_non_null_series = df.count(axis=aggregate_axis)
        mask = row_length == features_non_null_series
        
        return mask
    
    def missing_values_in_dataframe(df):
        row_length = len(df._get_axis(0))
        aggregate_axis = 1 - axis
        features_non_null_series = df.count(axis=aggregate_axis)
        mask = row_length == features_non_null_series
        print(df[mask[mask == 0].index.values].isnull().sum())
        print('\n')

In [3]:
train = suicides.df
test = suicides.df_test

In [4]:

## Region 적용 했을 때 성능이 더 높아짐 !! 429 --> 373  같은 country 별로 돌리는건데 왜 높아지는지는 모름



Europe = ["Albania","Russian Federation","France","Ukraine","Germany","Poland","United Kingdom",
         "Italy","Spain","Hungary","Romania","Belgium","Belarus","Netherlands","Austria",
         "Czech Republic","Sweden","Bulgaria","Finland","Lithuania","Switzerland","Serbia",
         "Portugal","Croatia","Norway","Denmark","Slovakia","Latvia","Greece","Slovenia",
         "Turkey","Estonia","Georgia","Albania","Luxembourg","Armenia","Iceland","Montenegro",
         "Cyprus","Bosnia and Herzegovina","San Marino","Malta","Ireland"]
NorthAmerica = ["United States","Mexico","Canada","Cuba","El Salvador","Puerto Rico",
                "Guatemala","Costa Rica","Nicaragua","Belize","Jamaica"]
SouthAmerica = ["Brazil","Colombia", "Chile","Ecuador","Uruguay","Paraguay","Argentina",
                "Panama","Guyana","Suriname"]
MiddleEast = ["Kazakhstan","Uzbekistan","Kyrgyzstan","Israel","Turkmenistan","Azerbaijan",
              "Kuwait","United Arab Emirates","Qatar","Bahrain","Oman"]
Asia = ["Japan","Republic of Korea", "Thailand", "Sri Lanka","Philippines","New Zealand",
        "Australia","Singapore","Macau","Mongolia"]

def match_region(x):
    if x in Europe:
        x = 'Europe'
    elif x in NorthAmerica:
        x = 'North America'
    elif x in SouthAmerica:
        x = 'South America'
    elif x in MiddleEast:
        x = 'MiddleEast'
    elif x in Asia:
        x = 'Asia'
    else:
        x = 'Island Nation'
    return x
  
  
train['region'] = train.country.map(lambda x: match_region(x))
  
test['region'] = test.country.map(lambda x: match_region(x))

In [5]:
categorical_feature = ['sex','age','region']


In [6]:
suicides.encode_labels_in_numeric_format(train, categorical_feature[0])
suicides.encode_labels_in_numeric_format(train, categorical_feature[1])
suicides.encode_labels_in_numeric_format(train, categorical_feature[2])
#suicides.encode_labels_in_numeric_format(train, categorical_feature[3])

suicides.encode_labels_in_numeric_format(test, categorical_feature[0])
suicides.encode_labels_in_numeric_format(test, categorical_feature[1])
suicides.encode_labels_in_numeric_format(test, categorical_feature[2])
#suicides.encode_labels_in_numeric_format(test, categorical_feature[3])

train.head()

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,...,Healthy life expectancy at birth,Positive affect,Negative affect,Democratic Quality,Life Ladder,Generosity,region,sex_count,age_count,region_count
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,0.635,2.156625e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,0.635,2.156625e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,0,1,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,0.635,2.156625e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,1,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,0.635,2.156625e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,0,2,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,0.635,2.156625e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,0,3,0


In [7]:
suicides.one_hot_encoder(train, categorical_feature[0])
suicides.one_hot_encoder(train, categorical_feature[1])
suicides.one_hot_encoder(train, categorical_feature[2])
#suicides.one_hot_encoder(train, categorical_feature[3])



suicides.one_hot_encoder(test, categorical_feature[0])
suicides.one_hot_encoder(test, categorical_feature[1])
suicides.one_hot_encoder(test, categorical_feature[2])
#suicides.one_hot_encoder(test, categorical_feature[3])
train.head()

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,...,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,0.635,2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,0.635,2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,0.635,2.156625e+09,...,0,0,0,0,1,0,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,0.635,2.156625e+09,...,1,0,0,0,1,0,0,0,0,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,0.635,2.156625e+09,...,0,1,0,0,1,0,0,0,0,0


In [8]:
train = train.drop(categorical_feature[0], axis=1)
train = train.drop(categorical_feature[1], axis=1)
train = train.drop(categorical_feature[2], axis=1)
#train = train.drop(categorical_feature[3], axis=1)
train = train.drop('country_year', axis=1)


test = test.drop(categorical_feature[0], axis=1)
test = test.drop(categorical_feature[1], axis=1)
test = test.drop(categorical_feature[2], axis=1)
#test = test.drop(categorical_feature[3], axis=1)
test = test.drop('country_year', axis=1)


train.head()

,country,year,suicides_no,population,suicides_100k_pop,HDI_for_year,gdp_for_year,gdp_per_capital,generation,Social support,...,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
0,Albania,1987,21,312900,6.71,0.635,2.156625e+09,796,Generation X,NaN,...,0,0,0,0,1,0,0,0,0,0
1,Albania,1987,16,308000,5.19,0.635,2.156625e+09,796,Silent,NaN,...,0,0,0,0,1,0,0,0,0,0
2,Albania,1987,14,289700,4.83,0.635,2.156625e+09,796,Generation X,NaN,...,0,0,0,0,1,0,0,0,0,0
3,Albania,1987,1,21800,4.59,0.635,2.156625e+09,796,G.I. Generation,NaN,...,1,0,0,0,1,0,0,0,0,0
4,Albania,1987,9,274300,3.28,0.635,2.156625e+09,796,Boomers,NaN,...,0,1,0,0,1,0,0,0,0,0


In [9]:

# 같은 조건일 때 2007년 : 399 / 2008년 : 339  / 2009년 : 348 / 2010 년 이후 : 381 -->  2008년 고정


train = train[train['year']> 2008]

## PreProcessing ( Feature Engineering )

In [10]:

# 필요없는 컬럼 삭제 . Generation 은 자살률과 무관한것으로 판단. 성능이 나빠진다. 

# suicides_100k_pop 필요. 이건 Train 데이터에서 파생변수로 만들어서 사용한다고 가정.  

# 보아즈 공지사항에 suicides_100k_pop 삭제하라고 했으니 삭제 후 population과 suicides_no 이용해서 만들자

test = test.drop(['suicides_100k_pop'], axis=1)
train = train.drop(['suicides_100k_pop'], axis=1)


train['suicides_100k_pop'] = train['suicides_no'] / train['population'] * 100000
 

  # generation 다시 추가도 해보고 새로운 컬럼들 이것저것 삭제도 해봤는데 이 상태가 그나마 최상
features_drop_train = ['suicides_no','sex_count','generation','age_count','region_count']
features_drop_test = ['sex_count','age_count','generation','region_count']

test = test.drop(features_drop_test, axis=1)
train = train.drop(features_drop_train, axis=1)


In [11]:
con = []
for i in test.country.unique():
    con.append(train[train.country == i])

In [12]:
train = pd.concat(con, axis=0)
print(train.head())
print(test.head())

                 country  year  population  HDI_for_year  gdp_for_year  \
528  Antigua and Barbuda  2009        8199          0.77  1.224253e+09   
529  Antigua and Barbuda  2009        6917          0.77  1.224253e+09   
530  Antigua and Barbuda  2009       14629          0.77  1.224253e+09   
531  Antigua and Barbuda  2009        8514          0.77  1.224253e+09   
532  Antigua and Barbuda  2009        4793          0.77  1.224253e+09   

     gdp_per_capital  Social support  Healthy life expectancy at birth  \
528            14335             NaN                               NaN   
529            14335             NaN                               NaN   
530            14335             NaN                               NaN   
531            14335             NaN                               NaN   
532            14335             NaN                               NaN   

     Positive affect  Negative affect  ...  age_25-34 years  age_55-74 years  \
528              NaN          

In [13]:
train[train.country == 'Antigua and Barbuda'] = train[train.country == 'Antigua and Barbuda'].fillna(0)
test[test.country == 'Antigua and Barbuda'] = test[test.country == 'Antigua and Barbuda'].fillna(0)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')
train.isna().sum(), test.isna().sum()

(country                             0
 year                                0
 population                          0
 HDI_for_year                        0
 gdp_for_year                        0
 gdp_per_capital                     0
 Social support                      0
 Healthy life expectancy at birth    0
 Positive affect                     0
 Negative affect                     0
 Democratic Quality                  0
 Life Ladder                         0
 Generosity                          0
 sex_male                            0
 sex_female                          0
 age_15-24 years                     0
 age_35-54 years                     0
 age_75+ years                       0
 age_25-34 years                     0
 age_55-74 years                     0
 age_5-14 years                      0
 region_Europe                       0
 region_Island Nation                0
 region_South America                0
 region_Asia                         0
 region_MiddleEast       

In [14]:
# 정규화

train_test_data=[train,test]
def normalize(X):
    return np.abs((X-np.mean(X))/np.std(X))
for dataset in train_test_data:
    dataset['gdp_per_capital']=normalize(dataset['gdp_per_capital'])
    dataset['gdp_for_year']=normalize(dataset['gdp_for_year'])
test.head()

,country,year,population,HDI_for_year,gdp_for_year,gdp_per_capital,Social support,Healthy life expectancy at birth,Positive affect,Negative affect,...,age_25-34 years,age_35-54 years,age_5-14 years,age_75+ years,region_Island Nation,region_South America,region_Europe,region_Asia,region_North America,region_MiddleEast
0,Antigua and Barbuda,2015,6403,0.785,0.32857,0.4737,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,Antigua and Barbuda,2015,8561,0.785,0.32857,0.4737,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,Antigua and Barbuda,2015,7740,0.785,0.32857,0.4737,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,0
3,Antigua and Barbuda,2015,15323,0.785,0.32857,0.4737,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,0
4,Antigua and Barbuda,2015,8239,0.785,0.32857,0.4737,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,0,0,0


In [15]:
# train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test1.csv')

##  Modeling


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet, ElasticNetCV
from pandas import Series, DataFrame
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor  


df_predict = DataFrame()
 
for i in test['country'].unique():
  
  train_by_country = train[train['country']==i]
  
  X = train_by_country.drop(['suicides_100k_pop','country','population'],axis=1) #independent columns
  y = train_by_country['suicides_100k_pop']

  # SPLIT할 때 TEST SIZE 0.01, random_state =100  로 할 때 성능 제일 좋다.   
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .01, random_state = 100)


  if (i== "United States"  or i== "Republic of Korea" or i== "Japan" or i== "Russian Federation" or i == 'Poland' or i == 'Kazakhstan'): 
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.9).fit(X_train, y_train)
    print(model.feature_importances_)
    
  elif (i == "Ukraine" or i == "Hungary" or i == "Brazil" or i == "Thailand" or i == "Mexico" or i == "Ecuador" or \
        i == "Colombia" or i == "Germany" or i == "Argentina" or i == "Israel" or i == "Italy" or i == "Australia" or \
        i == "Czech Republic" or i == "Romania"):
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.4, subsample=.9).fit(X_train, y_train)
    #print(model.feature_importances_)
  else:
    #model  = GradientBoostingRegressor(learning_rate=0.5).fit(X_train, y_train)
    #model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.7).fit(X_train, y_train)
    model = XGBRegressor(learning_rate=0.4, reg_lambda=.5, subsample=.9).fit(X_train, y_train)
    #print(model.feature_importances_)
    
    
  test_by_country = test[test['country']==i]

  test_by_country['generation_G.I. Generation']=0
  


  test_by_country_2= test_by_country.drop(['population','country'],axis=1)
  test_by_country_2 = test_by_country_2.reindex(columns = X.columns)


  ss= pd.Series(model.predict(test_by_country_2)).to_frame('predict')



  test_by_country=test_by_country.reset_index()
  test_result =pd.concat([test_by_country , pd.Series(model.predict(test_by_country_2)).to_frame('predict')] , axis=1)

  test_result['predicted_suicides_no']= test_result['predict'] /100000 * test_result['population']
  test_result= test_result.loc[:, ['country', 'year','predict','population','predicted_suicides_no']]

  df_predict=df_predict.append(test_result)

df_predict




[0.21921922 0.         0.03153153 0.         0.04504504 0.01801802
 0.03303303 0.02552553 0.         0.         0.0045045  0.22972973
 0.         0.07507508 0.08108108 0.05405406 0.03903904 0.07957958
 0.06456456 0.         0.         0.         0.         0.
 0.        ]
[0.10892587 0.         0.01512859 0.         0.03630862 0.03933434
 0.10590015 0.         0.02571861 0.         0.01815431 0.24357034
 0.         0.03630862 0.07866868 0.12859304 0.0484115  0.05295008
 0.06202723 0.         0.         0.         0.         0.
 0.        ]
[0.13023952 0.         0.0239521  0.         0.06437126 0.
 0.04640719 0.01946108 0.03443114 0.         0.01646707 0.26646706
 0.         0.06137725 0.05838323 0.06886227 0.0508982  0.11526946
 0.04341317 0.         0.         0.         0.         0.
 0.        ]
[0.18236174 0.         0.         0.01644245 0.03437967 0.
 0.06427503 0.06875934 0.         0.         0.         0.24813154
 0.         0.04484305 0.05231689 0.11509716 0.05979073 0.06576

,country,year,predict,population,predicted_suicides_no
0,Antigua and Barbuda,2015,0.000000,6403,0.000000
1,Antigua and Barbuda,2015,0.000000,8561,0.000000
2,Antigua and Barbuda,2015,0.000000,7740,0.000000
3,Antigua and Barbuda,2015,0.000000,15323,0.000000
4,Antigua and Barbuda,2015,0.000000,8239,0.000000
5,Antigua and Barbuda,2015,0.000000,1724,0.000000
6,Antigua and Barbuda,2015,0.000000,8331,0.000000
7,Antigua and Barbuda,2015,0.000000,7059,0.000000
8,Antigua and Barbuda,2015,0.000000,13370,0.000000
9,Antigua and Barbuda,2015,0.000000,8268,0.000000


In [17]:
X.head()

,year,HDI_for_year,gdp_for_year,gdp_per_capital,Social support,Healthy life expectancy at birth,Positive affect,Negative affect,Democratic Quality,Life Ladder,...,age_75+ years,age_25-34 years,age_55-74 years,age_5-14 years,region_Europe,region_Island Nation,region_South America,region_Asia,region_MiddleEast,region_North America
26592,2009,0.778,0.353059,0.673218,0.923861,67.337753,0.793431,0.254544,0.948057,6.296223,...,1,0,0,0,0,0,1,0,0,0
26593,2009,0.778,0.353059,0.673218,0.923861,67.337753,0.793431,0.254544,0.948057,6.296223,...,0,0,1,0,0,0,1,0,0,0
26594,2009,0.778,0.353059,0.673218,0.923861,67.337753,0.793431,0.254544,0.948057,6.296223,...,0,1,0,0,0,0,1,0,0,0
26595,2009,0.778,0.353059,0.673218,0.923861,67.337753,0.793431,0.254544,0.948057,6.296223,...,0,0,0,0,0,0,1,0,0,0
26596,2009,0.778,0.353059,0.673218,0.923861,67.337753,0.793431,0.254544,0.948057,6.296223,...,0,0,0,0,0,0,1,0,0,0


## Test

In [18]:
merge_data1 = df_predict['predicted_suicides_no'].groupby(df_predict['country']).sum()
merge_data2 =pd.DataFrame({'country':merge_data1.index, 'predicted_suicides_no':merge_data1.values})

merge_data2.to_csv('result_366.csv')

answer= pd.read_csv('./2015_suicide_no.csv')
answer.columns = ['country', 'suicides_no']

merge_answer= pd.merge(merge_data2, answer , on ='country', how='left')
merge_answer['error']=merge_answer['suicides_no']-merge_answer['predicted_suicides_no']
merge_answer['error_Abs']=abs(merge_answer['error'])
merge_answer1=merge_answer.sort_values('error_Abs',ascending=False)
merge_answer1

,country,predicted_suicides_no,suicides_no,error,error_Abs
45,Russian Federation,27092.801995,25432,-1660.801995,1660.801995
27,Japan,24398.116514,23092,-1306.116514,1306.116514
60,United States,43085.562689,44189,1103.437311,1103.437311
40,Poland,6137.929563,5420,-717.929563,717.929563
43,Republic of Korea,14069.349544,13510,-559.349544,559.349544
1,Argentina,3556.316624,3073,-483.316624,483.316624
58,Ukraine,8016.780364,7574,-442.780364,442.780364
55,Thailand,3861.620564,4205,343.379436,343.379436
15,Ecuador,757.577710,1073,315.422290,315.422290
52,Spain,3883.885124,3604,-279.885124,279.885124


In [19]:
true_value = merge_answer1.suicides_no
prediction_value = merge_answer1.predicted_suicides_no

print(np.sqrt(mean_squared_error(prediction_value, true_value)))

# 성능 339 -> 첫번째  XGBRegressor 가 learning_rate=0.4, reg_lambda=.5, subsample=.8 이고  
              #나머지 XGBRegressor 가 learning_rate=0.4, reg_lambda=.5, subsample=.7 일때. 
  

349.3767345081109
